In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [2]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/bellybutton.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Samples_Metadata = Base.classes.sample_metadata
Samples = Base.classes.samples

C:\Users\llreb\Anaconda3\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
sample = 940
sel = [
        Samples_Metadata.sample,
        Samples_Metadata.ETHNICITY,
        Samples_Metadata.GENDER,
        Samples_Metadata.AGE,
        Samples_Metadata.LOCATION,
        Samples_Metadata.BBTYPE,
        Samples_Metadata.WFREQ,
    ]

results = db.session.query(*sel).filter(Samples_Metadata.sample == sample).all()

# Create a dictionary entry for each row of metadata information
sample_metadata = {}
for result in results:
    sample_metadata["sample"] = result[0]
    sample_metadata["ETHNICITY"] = result[1]
    sample_metadata["GENDER"] = result[2]
    sample_metadata["AGE"] = result[3]
    sample_metadata["LOCATION"] = result[4]
    sample_metadata["BBTYPE"] = result[5]
    sample_metadata["WFREQ"] = result[6]

print(sample_metadata)

{'sample': 940, 'ETHNICITY': 'Caucasian', 'GENDER': 'F', 'AGE': 24.0, 'LOCATION': 'Beaufort/NC', 'BBTYPE': 'I', 'WFREQ': 2.0}


In [4]:
#Return `otu_ids`, `otu_labels`,and `sample_values`
#bubble chart data
stmt = db.session.query(Samples).statement
df = pd.read_sql_query(stmt, db.session.bind)
df.head()


,otu_id,otu_label,940,941,943,944,945,946,947,948,...,1562,1563,1564,1572,1573,1574,1576,1577,1581,1601
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Filter the data based on the sample number and
# only keep rows with values above 1
sample = "940"
sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
# Format the data to send as json
sample_data.head()

bubble_data = {
        "otu_ids": sample_data.otu_id.values.tolist,
        "sample_values": sample_data[sample].values.tolist,
        "otu_labels": sample_data.otu_label.values.tolist,
}
#data

In [ ]:
#Return `otu_ids`, `otu_labels`,and `sample_values`
#pie chart data
#stmt = db.session.query(Samples).statement
#df_pie = pd.read_sql_query(stmt, db.session.bind)
#df_pie.head()

In [ ]:
#sample = "940"
#sample_pie = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
# Format the data to send as json
#sample_pie.head()

In [6]:
sorted_df = sample_data.sort_values([sample], ascending = False)
sorted_df.head(10)

,otu_id,otu_label,940
1166,1167,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...,163
2858,2859,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,126
481,482,Bacteria,113
2263,2264,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,78
40,41,Bacteria,71
1188,1189,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...,51
351,352,Bacteria,50
188,189,Bacteria,47
2317,2318,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,40
1976,1977,Bacteria;Firmicutes;Clostridia;Clostridiales,40


In [7]:
sorted_pie = sorted_df.iloc[:10]
sorted_pie

,otu_id,otu_label,940
1166,1167,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...,163
2858,2859,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,126
481,482,Bacteria,113
2263,2264,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,78
40,41,Bacteria,71
1188,1189,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...,51
351,352,Bacteria,50
188,189,Bacteria,47
2317,2318,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,40
1976,1977,Bacteria;Firmicutes;Clostridia;Clostridiales,40


In [8]:
pie_data = {
        "otu_ids": sorted_pie.otu_id.values.tolist,
        "sample_values": sorted_pie[sample].values.tolist,
        "otu_labels": sorted_pie.otu_label.values.tolist,
}